<a href="https://colab.research.google.com/github/prishanmu/Music_Recommender/blob/master/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Artist Recommendation System

Inspired by a [Netflix recommendation system](https://www.kaggle.com/niharika41298/netflix-visualizations-recommendation-eda/notebook#Recommendation-System-(Content-Based)) I decided to create a similar recommendation system for music. 

In [0]:
import pandas as pd
import numpy as np

## Data 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import glob

path = "/content/drive/My Drive/Music Recommender/azlyrics-scraper/"

filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
  x = pd.read_csv(filename, header=None, sep='\n')
  x = x[0].str.split(',', expand=True)
  dfs.append(x)

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

In [0]:
big_frame.head()

In [0]:
#make first row header
df = big_frame
df.columns = df.iloc[0]

In [0]:
#recombine lyrics columns
cols_to_combine = list(df.columns) 

removal_list = ['ARTIST_NAME', '"ARTIST_URL"',  '"SONG_NAME"',   '"SONG_URL"']

for i in removal_list: 
  cols_to_combine.remove(i)
print(cols_to_combine[0:5]) #check

df['lyrics'] = df[cols_to_combine].agg(','.join, axis=1)


['"LYRICS"', None, None, None, None]


## TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

#removing stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['description'] = df['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

## Cosine Similarity

In [0]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
indices = pd.Series(netflix_overall.index, index=df['title']).drop_duplicates()

## Recomemmendation Function


In [0]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

## Getting Recommendatons

In [0]:
get_recommendation('title')